In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=aafa6acfd49224909c195440fa6a553d87461b30199ae3e71db8dcc24d71dcf8
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# **Spark Example**

In [ ]:
spark = SparkSession.builder \
      .appName("PySpark Notebook Example") \
      .getOrCreate()

In [ ]:
customers = [
    (1, "Ravi", "Mumbai"),
    (2, "Priya", "Delhi"),
    (3, "Vijay", "Bangalore"),
    (4, "Anita", "Chennai"),
    (5, "Raj", "Hyderabad"),
]

transactions = [
    (1, 1, 10000.50),
    (2, 2, 20000.75),
    (3, 1, 15000.25),
    (4, 3, 30000.00),
    (5, 2, 40000.50),
    (6, 4, 25000.00),
    (7, 5, 18000.75),
    (8, 1, 5000.00),
]

customer_col = ["customer_id", "Name", "city"]
transaction_col = ["transaction_id", "customer_id", "amount"]

customer_df = spark.createDataFrame(customers, schema = customer_col)
transaction_df = spark.createDataFrame(transactions, schema = transaction_col)

In [ ]:
print("Customer DataFrame:")
customer_df.show()

print("Transaction DataFrame:")
transaction_df.show()


Customer DataFrame:
+-----------+-----+---------+
|customer_id| Name|     city|
+-----------+-----+---------+
|          1| Ravi|   Mumbai|
|          2|Priya|    Delhi|
|          3|Vijay|Bangalore|
|          4|Anita|  Chennai|
|          5|  Raj|Hyderabad|
+-----------+-----+---------+

Transaction DataFrame:
+--------------+-----------+--------+
|transaction_id|customer_id|  amount|
+--------------+-----------+--------+
|             1|          1| 10000.5|
|             2|          2|20000.75|
|             3|          1|15000.25|
|             4|          3| 30000.0|
|             5|          2| 40000.5|
|             6|          4| 25000.0|
|             7|          5|18000.75|
|             8|          1|  5000.0|
+--------------+-----------+--------+



In [ ]:
# Join the DataFrames on CustomerID
customer_transactions_df = customer_df.join(transaction_df, on="customer_id")
print("Customer Transactions DataFrame:")
customer_transactions_df.show()

Customer Transactions DataFrame:
+-----------+-----+---------+--------------+--------+
|customer_id| Name|     city|transaction_id|  amount|
+-----------+-----+---------+--------------+--------+
|          1| Ravi|   Mumbai|             1| 10000.5|
|          1| Ravi|   Mumbai|             3|15000.25|
|          1| Ravi|   Mumbai|             8|  5000.0|
|          2|Priya|    Delhi|             2|20000.75|
|          2|Priya|    Delhi|             5| 40000.5|
|          3|Vijay|Bangalore|             4| 30000.0|
|          4|Anita|  Chennai|             6| 25000.0|
|          5|  Raj|Hyderabad|             7|18000.75|
+-----------+-----+---------+--------------+--------+



In [ ]:
# Calculate the total amount spent by each customer
total_spent_df = customer_transactions_df.groupBy ("Name").sum("Amount").withColumnRenamed ("sum(Amount)", "TotalSpent")
print("Total Amount Spent by Each Customer: ")
total_spent_df.show()

Total Amount Spent by Each Customer: 
+-----+----------+
| Name|TotalSpent|
+-----+----------+
| Ravi|  30000.75|
|Priya|  60001.25|
|Vijay|   30000.0|
|Anita|   25000.0|
|  Raj|  18000.75|
+-----+----------+



In [ ]:
# Find customers who have spent more than 30,000
big_spenders_df = total_spent_df.filter(col ("TotalSpent") > 30000)
print("Customers Who Spent More Than 30,000:")
big_spenders_df.show()

Customers Who Spent More Than 30,000:
+-----+----------+
| Name|TotalSpent|
+-----+----------+
| Ravi|  30000.75|
|Priya|  60001.25|
+-----+----------+



In [ ]:
# Count the number of transactions per customer
transactions_count_df = customer_transactions_df.groupBy ("Name").count().withColumnRenamed ("count", "TransactionCount")
print("Number of Transactions Per Customer: ")
transactions_count_df.show()

Number of Transactions Per Customer: 
+-----+----------------+
| Name|TransactionCount|
+-----+----------------+
| Ravi|               3|
|Priya|               2|
|Vijay|               1|
|Anita|               1|
|  Raj|               1|
+-----+----------------+



In [ ]:
# Sort customers by total amount spent in descending order
sorted_spenders_df = total_spent_df.orderBy(col ("TotalSpent").desc())
print ("Customers Sorted by Total Spent (Descending):")
sorted_spenders_df.show()

Customers Sorted by Total Spent (Descending):
+-----+----------+
| Name|TotalSpent|
+-----+----------+
|Priya|  60001.25|
| Ravi|  30000.75|
|Vijay|   30000.0|
|Anita|   25000.0|
|  Raj|  18000.75|
+-----+----------+



# **Exercise**

In [ ]:
# Exercise: Product Sales Analysis

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("Product Sales Analysis") \
    .getOrCreate()

products = [
    (1, "Laptop", "Electronics", 50000),
    (2, "Smartphone", "Electronics", 30000),
    (3, "Table", "Furniture", 15000),
    (4, "Chair", "Furniture", 5000),
    (5, "Headphones", "Electronics", 2000),
]

sales = [
    (1, 1, 2),
    (2, 2, 1),
    (3, 3, 3),
    (4, 1, 1),
    (5, 4, 5),
    (6, 2, 2),
    (7, 5, 10),
    (8, 3, 1),
]

product_columns = ["ProductID", "ProductName", "Category", "Price"]
sales_columns = ["SaleID", "ProductID", "Quantity"]

product_df = spark.createDataFrame(products, schema=product_columns)
sales_df = spark.createDataFrame(sales, schema=sales_columns)

print("Products DataFrame:")
product_df.show()

print("Sales DataFrame:")
sales_df.show()

Products DataFrame:
+---------+-----------+-----------+-----+
|ProductID|ProductName|   Category|Price|
+---------+-----------+-----------+-----+
|        1|     Laptop|Electronics|50000|
|        2| Smartphone|Electronics|30000|
|        3|      Table|  Furniture|15000|
|        4|      Chair|  Furniture| 5000|
|        5| Headphones|Electronics| 2000|
+---------+-----------+-----------+-----+

Sales DataFrame:
+------+---------+--------+
|SaleID|ProductID|Quantity|
+------+---------+--------+
|     1|        1|       2|
|     2|        2|       1|
|     3|        3|       3|
|     4|        1|       1|
|     5|        4|       5|
|     6|        2|       2|
|     7|        5|      10|
|     8|        3|       1|
+------+---------+--------+



In [ ]:
# 1.Join the DataFrames:
# Join the product_df and sales_df DataFrames on ProductID to create a combined DataFrame with product and sales data.
product_sales_df = product_df.join(sales_df, on="ProductID")
print("product_sales DataFrame:")
product_sales_df.show()

product_sales DataFrame:
+---------+-----------+-----------+-----+------+--------+
|ProductID|ProductName|   Category|Price|SaleID|Quantity|
+---------+-----------+-----------+-----+------+--------+
|        1|     Laptop|Electronics|50000|     1|       2|
|        1|     Laptop|Electronics|50000|     4|       1|
|        2| Smartphone|Electronics|30000|     2|       1|
|        2| Smartphone|Electronics|30000|     6|       2|
|        3|      Table|  Furniture|15000|     3|       3|
|        3|      Table|  Furniture|15000|     8|       1|
|        4|      Chair|  Furniture| 5000|     5|       5|
|        5| Headphones|Electronics| 2000|     7|      10|
+---------+-----------+-----------+-----+------+--------+



In [ ]:
# 2.Calculate Total Sales Value:
# For each product, calculate the total sales value by multiplying the price by the quantity sold.

total_sale_product_df = product_sales_df.withColumn("TotalSalesValue", col("Price") * col("Quantity"))
print("Total Sales Value DataFrame:")
total_sale_product_df.show()

Total Sales Value DataFrame:
+---------+-----------+-----------+-----+------+--------+---------------+
|ProductID|ProductName|   Category|Price|SaleID|Quantity|TotalSalesValue|
+---------+-----------+-----------+-----+------+--------+---------------+
|        1|     Laptop|Electronics|50000|     1|       2|         100000|
|        1|     Laptop|Electronics|50000|     4|       1|          50000|
|        2| Smartphone|Electronics|30000|     2|       1|          30000|
|        2| Smartphone|Electronics|30000|     6|       2|          60000|
|        3|      Table|  Furniture|15000|     3|       3|          45000|
|        3|      Table|  Furniture|15000|     8|       1|          15000|
|        4|      Chair|  Furniture| 5000|     5|       5|          25000|
|        5| Headphones|Electronics| 2000|     7|      10|          20000|
+---------+-----------+-----------+-----+------+--------+---------------+



In [ ]:
# 3.Find the Total Sales for Each Product Category:
# Group the data by the Category column and calculate the total sales value for each product category.
total_sale_by_category_df = total_sale_product_df.groupBy("Category").sum("TotalSalesValue").withColumnRenamed("sum(TotalSalesValue)","TotalSales")
print("Total Sales for Each Product Category:")
total_sale_by_category_df.show()

Total Sales for Each Product Category:
+-----------+----------+
|   Category|TotalSales|
+-----------+----------+
|Electronics|    260000|
|  Furniture|     85000|
+-----------+----------+



In [ ]:
# 4.Identify the Top-Selling Product:
# Find the product that generated the highest total sales value.
high_sale_product = total_sale_product_df.groupBy("ProductName").sum("TotalSalesValue").withColumnRenamed("sum(TotalSalesValue)","TotalSales").orderBy(col("TotalSales").desc()).limit(1)
print("Top-Selling Product:")
high_sale_product.show()


Top-Selling Product:
+-----------+----------+
|ProductName|TotalSales|
+-----------+----------+
|     Laptop|    150000|
+-----------+----------+



In [ ]:
# 5.Sort the Products by Total Sales Value:
# Sort the products by total sales value in descending order.
high_sale_product = total_sale_product_df.groupBy("ProductName").sum("TotalSalesValue").withColumnRenamed("sum(TotalSalesValue)","TotalSales").orderBy(col("TotalSales").desc())
print("product's Total sales value")
high_sale_product.show()


product's Total sales value
+-----------+----------+
|ProductName|TotalSales|
+-----------+----------+
|     Laptop|    150000|
| Smartphone|     90000|
|      Table|     60000|
|      Chair|     25000|
| Headphones|     20000|
+-----------+----------+



In [ ]:
# 6.Count the Number of Sales for Each Product:
# Count the number of sales transactions for each product.
product_sales_count_df = product_sales_df.groupBy("ProductID").count().withColumnRenamed("count","TransactionCount")
print("Number of Sales for Each Product:")
product_sales_count_df.show()

Number of Sales for Each Product:
+---------+----------------+
|ProductID|TransactionCount|
+---------+----------------+
|        1|               2|
|        2|               2|
|        3|               2|
|        4|               1|
|        5|               1|
+---------+----------------+



In [ ]:
# 7.Filter the Products with Total Sales Value Greater Than ₹50,000:
# Filter out the products that have a total sales value greater than ₹50,000.
filtered_high_sale_product = high_sale_product.filter(col("TotalSales") > 50000)
print("Products with Total Sales Value Greater Than ₹50,000:")
filtered_high_sale_product.show()


Products with Total Sales Value Greater Than ₹50,000:
+-----------+----------+
|ProductName|TotalSales|
+-----------+----------+
|     Laptop|    150000|
| Smartphone|     90000|
|      Table|     60000|
+-----------+----------+



#**RDD Transformation** (Resilient Distributed Dataset)

In [ ]:
spark = SparkSession.builder \
      .appName("RDD Transformation Example") \
      .getOrCreate()

sc = spark.sparkContext
print("Spark Session Created")

Spark Session Created


In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd = sc.parallelize(data)

print("original RDD:", rdd.collect())

original RDD: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
rdd2 = rdd.map(lambda x: x * 2)

print("RDD after transformation (x**2):", rdd2.collect())

RDD after transformation (x**2): [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [ ]:
rdd3 = rdd2.filter(lambda x: x % 2 == 0)

print("RDD after filtering (even):", rdd3.collect())

RDD after filtering (even): [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [ ]:
sentence = ["Hello World","py spark is geat", "RDD transformations"]
rdd4 = sc.parallelize(sentence)
ScentenceToWords_rdd = rdd4.flatMap(lambda x: x.split(" "))

print("RDD after flatMap transformation:", ScentenceToWords_rdd.collect())

RDD after flatMap transformation: ['Hello', 'World', 'py', 'spark', 'is', 'geat', 'RDD', 'transformations']


In [ ]:
# Actions
# 1. collect()
result = rdd.collect()
print("Result of collect action:", result)
# 2. count()
result = rdd.count()
print("Result of count action:", result)
# 3. reduce()
result = rdd.reduce(lambda x, y: x + y)
print("Result of reduce action:", result)

Result of collect action: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Result of count action: 10
Result of reduce action: 55


# **RDD Exercise**

In [ ]:
# https://codeshare.io/ez3VNJ
# Initialize SparkSession
spark = SparkSession.builder.appName("SalesDataAnalysis").getOrCreate()

In [ ]:
sales_data = [
    ("ProductA", 100),
    ("ProductB", 150),
    ("ProductA", 200),
    ("ProductC", 300),
    ("ProductB", 250),
    ("ProductC", 100)
]

# step 1 -> spark context
sc = spark.sparkContext

# step 2
# task 1 -> creating RDD of sales_data and Printing the first few elements of the RDD
sales_rdd = sc.parallelize(sales_data)
print(sales_rdd.take(3))

[('ProductA', 100), ('ProductB', 150), ('ProductA', 200)]


In [ ]:
# step 3 -> Grouping and Aggregating Data

# taks 2 -> Group data by product name
grouped_sales_rdd = sales_rdd.groupByKey()
print("Grouped data:")
for k,v in grouped_sales_rdd.collect():
  print(k,list(v))

# taks 3 -> Calculate total sales by product
total_sales_by_product = sales_rdd.reduceByKey(lambda x, y: x + y)
print("Total sales by product:")
print(total_sales_by_product.collect())

# taks 4 -> Sort products by total sales
sorted_products = total_sales_by_product.sortBy(lambda x: x[1], ascending=False)
print("Sorted products by total sales:")
print(sorted_products.collect())



Grouped data:
ProductA [100, 200]
ProductB [150, 250]
ProductC [300, 100]
Total sales by product:
[('ProductA', 300), ('ProductB', 400), ('ProductC', 400)]
Sorted products by total sales:
[('ProductB', 400), ('ProductC', 400), ('ProductA', 300)]


In [ ]:
#step 4 -> Additional transformations

# taks 5 -> Filter products with high sales
high_sales_products = total_sales_by_product.filter(lambda x: x[1] > 300)
print("Products with high sales:")
print(high_sales_products.collect())

# task 6 -> Combine Regional Sales Data

# regional sales data RDD
regional_sales_data = [
    ("ProductA", 50),
    ("ProductC", 150)
]
regional_sales_rdd = sc.parallelize(regional_sales_data)

# Combining the two RDDs
combined_sales_rdd = sales_rdd.union(regional_sales_rdd)

# Calculating new total sales
new_total_sales_by_product = combined_sales_rdd.reduceByKey(lambda x, y: x + y)
print("Combined sales data:")
print(new_total_sales_by_product.collect())

Products with high sales:
[('ProductB', 400), ('ProductC', 400)]
Combined sales data:
[('ProductA', 350), ('ProductC', 550), ('ProductB', 400)]


In [ ]:
# step 5 ->  Perform Actions on the RDD

# task 7 -> Count the number of distinct products
distinct_products_count = sales_rdd.map(lambda x: x[0]).distinct().count()
print("Count of distinct products:", distinct_products_count)

# task 8 -> Identify the product with maximum sales
max_sales = total_sales_by_product.max()[1]
max_sales_products = total_sales_by_product.filter(lambda x: x[1] == max_sales)
print("Products with maximum sales:", max_sales_products.map(lambda x: x[0]).collect())


Count of distinct products: 3
Products with maximum sales: ['ProductB', 'ProductC']


In [ ]:
# challenge -> Calculate the Average Sales per Product
for k,v in grouped_sales_rdd.collect():
  print(k,sum(list(v))/len(list(v)))


ProductA 150.0
ProductB 200.0
ProductC 200.0


# **PySpark Exercise with Pandas**

In [4]:
# https://codeshare.io/w90yOJ

spark = SparkSession.builder \
    .appName("Employee Data Analysis") \
    .getOrCreate()

In [5]:
# Sample employee data
data = [
    (1, 'Arjun', 'IT', 75000),
    (2, 'Vijay', 'Finance', 85000),
    (3, 'Shalini', 'IT', 90000),
    (4, 'Sneha', 'HR', 50000),
    (5, 'Rahul', 'Finance', 60000),
    (6, 'Amit', 'IT', 55000)
]

# Define schema (columns)
columns = ['EmployeeID', 'EmployeeName', 'Department', 'Salary']

# Create DataFrame
employee_df = spark.createDataFrame(data, columns)

# Show the DataFrame
employee_df.show()

+----------+------------+----------+------+
|EmployeeID|EmployeeName|Department|Salary|
+----------+------------+----------+------+
|         1|       Arjun|        IT| 75000|
|         2|       Vijay|   Finance| 85000|
|         3|     Shalini|        IT| 90000|
|         4|       Sneha|        HR| 50000|
|         5|       Rahul|   Finance| 60000|
|         6|        Amit|        IT| 55000|
+----------+------------+----------+------+



In [6]:
# Task 1: Filter Employees by Salary

high_salary_employees = employee_df.filter(col("Salary") > 60000)
print("Employees with salary greater than 60000:")
high_salary_employees.show()

Employees with salary greater than 60000:
+----------+------------+----------+------+
|EmployeeID|EmployeeName|Department|Salary|
+----------+------------+----------+------+
|         1|       Arjun|        IT| 75000|
|         2|       Vijay|   Finance| 85000|
|         3|     Shalini|        IT| 90000|
+----------+------------+----------+------+



In [11]:
# Task 2: Calculate the Average Salary by Department

avg_salary_by_dept = employee_df.groupBy("Department").avg("Salary").withColumnRenamed("avg(Salary)", "AvgerageSalary")
print("Average salary by department:")
avg_salary_by_dept.show()

Average salary by department:
+----------+-----------------+
|Department|   AvgerageSalary|
+----------+-----------------+
|   Finance|          72500.0|
|        IT|73333.33333333333|
|        HR|          50000.0|
+----------+-----------------+



In [13]:
# Task 3: Sort Employees by Salary (Descending)

sorted_by_salary_desc = employee_df.orderBy(col("Salary").desc())
print("Employees sorted by salary descending:")
sorted_by_salary_desc.show()

Employees sorted by salary descending:
+----------+------------+----------+------+
|EmployeeID|EmployeeName|Department|Salary|
+----------+------------+----------+------+
|         3|     Shalini|        IT| 90000|
|         2|       Vijay|   Finance| 85000|
|         1|       Arjun|        IT| 75000|
|         5|       Rahul|   Finance| 60000|
|         6|        Amit|        IT| 55000|
|         4|       Sneha|        HR| 50000|
+----------+------------+----------+------+



In [14]:
# Task 4: Add a Bonus Column

employee_df_with_bonus = employee_df.withColumn("Bonus", col("Salary") * 0.1)
print("Employees with bonus column:")
employee_df_with_bonus.show()

Employees with bonus column:
+----------+------------+----------+------+------+
|EmployeeID|EmployeeName|Department|Salary| Bonus|
+----------+------------+----------+------+------+
|         1|       Arjun|        IT| 75000|7500.0|
|         2|       Vijay|   Finance| 85000|8500.0|
|         3|     Shalini|        IT| 90000|9000.0|
|         4|       Sneha|        HR| 50000|5000.0|
|         5|       Rahul|   Finance| 60000|6000.0|
|         6|        Amit|        IT| 55000|5500.0|
+----------+------------+----------+------+------+

